In [5]:
import sys
sys.path.append('../../../')
from QC5 import * 

import numpy as np
import matplotlib.pyplot as plt 
%config InlineBackend.figure_format = 'retina'
from pyscf import gto,scf,fci,mcscf,ao2mo

# R=2.0

In [79]:
# Optimized version for computing h1eff and h2act 
def get_ActiveSpaceModelOpt(mol, mf, mo_coeff, list_frozen, list_active):
    ncore = len(list_frozen)
    nact = len(list_active)
    # fake object
    nelecas = mol.nelectron - 2*ncore
    mc = mcscf.CASCI(mf, nact, nelecas)
    h1act, ecore = mc.get_h1eff(mo_coeff)
    mo_act = mo_coeff[:,list_active]
    h2act = ao2mo.outcore.general_iofree(mol,(mo_act,)*4,compact=0).reshape(nact,nact,nact,nact)
    return ecore,h1act,h2act

# molecule
mol = gto.Mole()
mol.atom = genAn('N',2,2.0)
mol.basis = 'sto3g' 
mol.charge = 0 
mol.spin = 0
mol.symmetry = True
mol.build()   
# RHF

mf = scf.RHF(mol)
eRHF = mf.kernel()


mo_coeff = mf.mo_coeff
ovlp = mf.get_ovlp()
list_frozen=[0,1,2,3]
list_active=[4,5,6,7,8,9]
ecore,h1a,h2a = get_ActiveSpaceModelOpt(mol,mf, mo_coeff, list_frozen, list_active)

norb  = len(list_active)
nocc = np.count_nonzero(mf.get_occ()) - len(list_frozen)
nelec = (nocc,nocc)

print(mf.get_occ())
print('active',list_active,'frozen',list_frozen)
print('new norb',norb,'new nelec',nelec)

# # RHF basis
fermion_transform = 'jordan_wigner'
h1,h2 = get_mo2so_h1_h2_int(h1a,h2a)
qubit_hamiltonian = get_qubit_hamiltonian(ecore,h1,h2,fermion_transform)

converged SCF energy = -106.871504045608
[2. 2. 2. 2. 2. 2. 2. 0. 0. 0.]
active [4, 5, 6, 7, 8, 9] frozen [0, 1, 2, 3]
new norb 6 new nelec (3, 3)


In [80]:
# na nb penalty ham
qubit_na,qubit_nb = get_qubit_na_nb_operator(norb,fermion_transform)
qubit_na_nb_preserve = (qubit_na-nelec[0])**2 + (qubit_nb-nelec[1])**2
qubit_splus,qubit_sminus = get_qubit_adder_operator(ovlp,mo_coeff,fermion_transform,list_active)
beta = 1.0
ham = qubit_hamiltonian + beta*qubit_na_nb_preserve + beta*qubit_splus*qubit_sminus